In [13]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd 
import random
import os

In [2]:
%matplotlib inline
plt.rcParams['figure.figsize'] = (5.0, 4.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

%load_ext autoreload
%autoreload 2

np.random.seed(1)

In [22]:
filenames = os.listdir("train/train")
for filename in filenames:
    try:
        name = filename.split(":")[1]
    except:
        pass
    if name == "Zone.Identifier":
        os.remove("train/train/" + filename)

In [23]:
filenames = os.listdir("train/train")
categories = []
for filename in filenames:
    category = filename.split('.')[0]
    if category == 'dog':
        categories.append(1)
    else:
        categories.append(0)

df = pd.DataFrame({
    'filename': filenames,
    'category': categories
})

In [24]:
df.head()


,filename,category
0,dog.9435.jpg,1
1,dog.539.jpg:Zone.Identifier,1
2,cat.6952.jpg,0
3,cat.1399.jpg,0
4,cat.10565.jpg:Zone.Identifier,0
